In [19]:
import os, json, re, getpass
from dotenv import load_dotenv
import pandas as pd
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [20]:
load_dotenv(".env", override=True)
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("GROQ API Key: ")

In [21]:

data = pd.read_csv("test_data.csv", nrows=25)
data.head()

,Name,Point1,Point2,Point3,Extra
0,Tanaya Verma,15/20,30/50,18/20,Had extraodinary participation in MUN.


In [22]:
model_id = "llama3-8b-8192" #llama3-8b-8192, llama-3.1-8b-instant, llama3-groq-8b-8192-tool-use-preview, llama3-groq-70b-8192-tool-use-preview
llm = ChatGroq(model_name=model_id, temperature=0)
system_message_template = """Provided is a set of points for a students performance both academic and otherwise. The first column is the name, 
second is the performance in the first unit test, third half yearly, fourth second unit test and last one is an extra comment the teacher has about the student. 
Create a review of the student using the given points similar to what a teacher would give. Give the review in a single paragraph under the name review as a json output only.
Provide the output which can be parsed by a json parser, not text.
"""
template = ChatPromptTemplate([
    ("system", system_message_template),
    ("human", "{user_input}"),])
review_chain = template | llm | JsonOutputParser()

In [23]:
arr = []
for index, row in data.iterrows():
    temp = row.to_dict()
    review = row
    retry_count = 5
    while retry_count >= 0:
        try:
            chain_output = review_chain.invoke(dict(
            user_input = review
            ))
        except Exception as err:
            print(f"Exception found for review {index}, Retrying: {retry_count}")
            print("Exception: ", str(err))
            retry_count -= 1
            continue
        break
    arr.append(temp|chain_output)
    print("review processed: ", index)

review processed:  0


In [24]:
out = pd.DataFrame(arr)
out.to_csv("output.csv", index=False)